In [1]:
import tensorflow as tf
import numpy as np
from tensorflow import keras

In [2]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()
#总计20640个样本，每个样本8个属性表示，以及房价作为target，所有属性值均为number
#目标变量：平均房屋价值
#输入变量（特征）：平均收入、住房平均年龄、平均房间、平均卧室、人口、平均占用、纬度和经度

X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target.reshape(-1, 1), random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)
X_test_scaled = scaler.transform(X_test)

In [5]:
input_shape = X_train.shape[1:]

In [3]:
#创建一个函数重新创建已配置的损失函数，阈值默认值为1.0
def create_huber(threshold=1.0):
    def huber_fn(y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < threshold
        squared_loss = tf.square(error) / 2
        linear_loss  = threshold * tf.abs(error) - threshold**2 / 2
        return tf.where(is_small_error, squared_loss, linear_loss)
    return huber_fn

### Custom Metrics自定义指标

In [6]:
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="selu", kernel_initializer="lecun_normal",
                       input_shape=input_shape),
    keras.layers.Dense(1),
])
model.compile(loss="mse", optimizer="nadam", metrics=[create_huber(2.0)])
#用huber损失作为一个指标

model.fit(X_train_scaled, y_train, epochs=2)

Epoch 1/2
363/363 [==============================] - 1s 884us/step - loss: 2.1071 - huber_fn: 0.9148
Epoch 2/2
363/363 [==============================] - 0s 820us/step - loss: 0.5010 - huber_fn: 0.2443


将huber损失定义为loss和metric出来的模型效果不一样

loss = metric × 样本权重平均值 (加上一些浮点精确值的误差)

loss：损失函数的目的是计算模型在训练过程中最小化的数值。实际的优化目标：所有数据点输出数组的平均值。

metric：指标是用于判断模型性能的函数，训练模型时不使用指标，可使用任何loss函数作为指标。

对训练期间的每一批次keras都会计算该指标、并自动跟踪自轮次开始以来的均值（但阈值无法保存）

In [7]:
model.compile(loss=create_huber(2.0), optimizer="nadam", 
              metrics=[create_huber(2.0)])
#若第一轮次指标为0.8，第二轮次为0，则第二轮次后两次指标均值得到的指标值为0.4
#与自轮次开始以来总体指标0.5不相等，不正确

sample_weight = np.random.rand(len(y_train))
#生成和训练集样本个数同样数量的权重随机值

history = model.fit(X_train_scaled, y_train, epochs=2, 
                    sample_weight=sample_weight)

Epoch 1/2
363/363 [==============================] - 1s 970us/step - loss: 0.1144 - huber_fn: 0.2337
Epoch 2/2
363/363 [==============================] - 0s 928us/step - loss: 0.1097 - huber_fn: 0.2262


model.fit()返回一个History对象，包含训练参数history.params、经历的轮次列表history.epoch，包含在训练集和验证集上每个轮次结束时测得的损失和格外指标的字典history.history。

In [8]:
history.history["loss"][0], history.history["huber_fn"][0] * sample_weight.mean()
#将metric得到的metric × 样本权重平均值得到的loss与自每个epoch开始的loss比较
#差距是由于浮点计算引起的误差值

(0.11435428261756897, 0.11704190442014657)

### 流式指标（逐批次更新的状态指标）

keras.metrics.Precision类：跟踪真正和假正的数量，在被请求时计算其比率

In [9]:
precision = keras.metrics.Precision()
precision([0, 1, 1, 1, 0, 1, 0, 1], 
          [1, 1, 0, 1, 0, 1, 0, 1])
#做了5次正预测其中4次是对的，本轮次精度ACC = 真正例/（真正例+假正例）= 4/5=80%
#由于是第一轮次，因此目前为止总体精度=0.8

<tf.Tensor: shape=(), dtype=float32, numpy=0.8>

In [10]:
precision([0, 1, 0, 0, 1, 0, 1, 0],
          [1, 0, 1, 1, 0, 0, 0, 0])
#做了3次正预测都不对，本轮次精度 =0%，
#目前为止总体精度=目前为止所有真正例/（目前为止所有真正例+目前为止所有假正例）=4/8=0.5

<tf.Tensor: shape=(), dtype=float32, numpy=0.5>

result()：获取指标当前值

In [11]:
precision.result()
#目前为止总体精度

<tf.Tensor: shape=(), dtype=float32, numpy=0.5>

variables属性:查看变量

In [12]:
precision.variables
#真正例和假正例判断次数都为4

[<tf.Variable 'true_positives:0' shape=(1,) dtype=float32, numpy=array([4.], dtype=float32)>,
 <tf.Variable 'false_positives:0' shape=(1,) dtype=float32, numpy=array([4.], dtype=float32)>]

reset_states():重置变量

In [13]:
precision.reset_states()
precision.variables#变量都被设置为0了

[<tf.Variable 'true_positives:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>,
 <tf.Variable 'false_positives:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>]

#### 创建流式指标

keras.add_weight():手动创建变量

tf.cast(x，dtype):类型转换

tf.assign_add(ref, value, use_locking=None, name=None)：将值 value 加到变量 ref 上

tf.size ()：返回输入张量的大小（元素个数）

In [14]:
class HuberMetric(keras.metrics.Metric):
    """
    继承keras.metrics.Metric类，跟踪Huber总损失（目前为止总体loss），
    keras在每个批次调用该指标，跟踪每轮次的均值，
    好处是：可保存阈值
    """
    def __init__(self, threshold=1.0, **kwargs):
        super().__init__(**kwargs) # handles base args (e.g., dtype)
        self.threshold = threshold
        self.huber_fn = create_huber(threshold)
        self.total = self.add_weight("total", initializer="zeros")
        #跟踪所有Huber损失的总和（总计）,初始化为0
        self.count = self.add_weight("count", initializer="zeros")
        #跟踪目前为止的所有实例数（计数）

    def update_state(self, y_true, y_pred, sample_weight=None):
        """给定一个批次的标签和预测值，更新变量"""
        metric = self.huber_fn(y_true, y_pred)
        #调用huber_add方法，得到所有标签被判定正确的loss
        self.total.assign_add(tf.reduce_sum(metric))
        self.count.assign_add(tf.cast(tf.size(y_true), tf.float32))  
        
    def result(self):
        """指标=目前为止所有批次的loss/目前为止所有实例"""
        return self.total / self.count
    
    def get_config(self):
        """确保threshold与模型一起被保存"""
        base_config = super().get_config()
        return {**base_config, "threshold": self.threshold}

In [15]:
m = HuberMetric(2.)
#创建HuberMetric类的实例m
#huber loss的阈值设为2.0

In [16]:
# error=10-2=8>2，总体loss：total = 2 * |10 - 2| - 2²/2 = 14
# count = 1，一共跟踪了一个实例
# result = 14 / 1 = 14

#y_true真实标签为2, y_pred预测标签为10
m(tf.constant([[2.]]), tf.constant([[10.]])) 

<tf.Tensor: shape=(), dtype=float32, numpy=14.0>

In [17]:
# error=（1-0<2）、（9.25-5>2）
#total = total + (|1 - 0|² / 2) + (2 * |9.25 - 5| - 2² / 2) = 14 + 7 = 21
# count = count + 2 = 3,两个轮次一共3个实例
# result = total / count = 21 / 3 = 7
m(tf.constant([[0.], [5.]]), tf.constant([[1.], [9.25]]))
m.result()

<tf.Tensor: shape=(), dtype=float32, numpy=7.0>

In [18]:
m.variables

[<tf.Variable 'total:0' shape=() dtype=float32, numpy=21.0>,
 <tf.Variable 'count:0' shape=() dtype=float32, numpy=3.0>]

In [19]:
m.reset_states()
m.variables

[<tf.Variable 'total:0' shape=() dtype=float32, numpy=0.0>,
 <tf.Variable 'count:0' shape=() dtype=float32, numpy=0.0>]

#### 测试HuberMetric类构建流式指标的模型效果

In [20]:
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="selu", 
                       kernel_initializer="lecun_normal",
                       input_shape=input_shape),
    keras.layers.Dense(1),
])
model.compile(loss=create_huber(2.0), optimizer="nadam", 
              metrics=[HuberMetric(2.0)])
#设置指标metrics为阈值2的HuberMetric实例

model.fit(X_train_scaled.astype(np.float32), 
          y_train.astype(np.float32), epochs=2)

Epoch 1/2
363/363 [==============================] - 1s 950us/step - loss: 0.7793 - huber_metric_1: 0.7793
Epoch 2/2
363/363 [==============================] - 0s 905us/step - loss: 0.2337 - huber_metric_1: 0.2337


In [21]:
model.save("my_model_with_a_custom_metric.h5")
model = keras.models.load_model("my_model_with_a_custom_metric.h5",
                                custom_objects={"huber_fn": create_huber(2.0),
                                                "HuberMetric": HuberMetric})
model.fit(X_train_scaled.astype(np.float32), 
          y_train.astype(np.float32), epochs=2)

Epoch 1/2
363/363 [==============================] - 1s 874us/step - loss: 0.2236 - huber_metric_1: 0.2236
Epoch 2/2
363/363 [==============================] - 0s 853us/step - loss: 0.2173 - huber_metric_1: 0.2173


In [22]:
model.metrics[-1].threshold
#最后一个指标的阈值

2.0

#### 创建流式指标（简化）

update_state():将每一次更新的数据作为一组数据，在真正计算的时候会计算出每一组的结果，然后求多组结果的平均值

In [23]:
class HuberMetric(keras.metrics.Mean):
    """
    继承keras.metrics.Mean类，
    结果为到目前为止所有epoch的loss平均值
    """
    def __init__(self, threshold=1.0, name='HuberMetric', dtype=None):
        self.threshold = threshold
        self.huber_fn = create_huber(threshold)
        super().__init__(name=name, dtype=dtype)
        
    def update_state(self, y_true, y_pred, sample_weight=None):
        """指标=目前为止所有批次的loss/目前为止所有实例"""
        metric = self.huber_fn(y_true, y_pred)
        #调用huber_add方法，得到标签被判定正确的loss
        super(HuberMetric, self).update_state(metric, sample_weight)
        #计算指标的平均值
        
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "threshold": self.threshold}        

model.compile（）的metrics参数：用于评估当前训练模型的性能的评价函数。当模型编译后评价函数应该作为 metrics 的参数来输入。评价函数的结果不会用于训练过程中。

model.compile（）的weighted_metrics参数：在训练和测试期间，要由 sample_weight 或 class_weight 评估和加权的指标列表,会给出一系列指标

In [24]:
#测试效果
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="selu", kernel_initializer="lecun_normal",
                       input_shape=input_shape),
    keras.layers.Dense(1),
])
model.compile(loss=keras.losses.Huber(2.0), 
              optimizer="nadam", 
              weighted_metrics=[HuberMetric(2.0)])
#设置权重指标列表为阈值为2的HuberMetric类实例

sample_weight = np.random.rand(len(y_train))
history = model.fit(X_train_scaled.astype(np.float32), y_train.astype(np.float32),
                    epochs=2, sample_weight=sample_weight)

Epoch 1/2
363/363 [==============================] - 1s 1ms/step - loss: 0.3993 - HuberMetric: 0.8024
Epoch 2/2
363/363 [==============================] - 0s 1ms/step - loss: 0.1232 - HuberMetric: 0.2475


In [25]:
history.history["loss"][0]

0.39927175641059875

In [26]:
#目前为止总体loss = 目前为止所有epoch的loss平均值metric × 样本权重的平均值 
#(加上一些浮点精确值的误差)
history.history["HuberMetric"][0] * sample_weight.mean()

0.39927155268077624

In [27]:
model.save("my_model_with_a_custom_metric_v2.h5")
model = keras.models.load_model("my_model_with_a_custom_metric_v2.h5",
                                custom_objects={"HuberMetric": HuberMetric})
model.fit(X_train_scaled.astype(np.float32), 
          y_train.astype(np.float32), epochs=2)

Epoch 1/2
363/363 [==============================] - 1s 907us/step - loss: 0.2250 - HuberMetric: 0.2250
Epoch 2/2
363/363 [==============================] - 0s 948us/step - loss: 0.2159 - HuberMetric: 0.2159


In [28]:
#最后一个指标（huber loss）的阈值
model.metrics[-1].threshold

2.0